In [ ]:
import random
import os, sys

import numpy as np
from torch.utils.data import Subset
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
from torch.utils.tensorboard import SummaryWriter

from sklearn.model_selection import StratifiedKFold

sys.path.append(os.path.abspath('..'))
from dataset import MaskBaseDataset
from model import *
from loss import create_criterion

sys.path.append('../')
from dataset import MaskMultiClassDataset

def seed_everything(seed):
    """
    동일한 조건으로 학습을 할 때, 동일한 결과를 얻기 위해 seed를 고정시킵니다.
    
    Args:
        seed: seed 정수값
    """
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
seed_everything(42)

## Lesson 9 - Ensemble
- 이번 실습 자료에서는 강의시간에 다루었던 Straified kFold Cross Validation(교차 검증)에 대해 다뤄보겠습니다. k개의 Fold로 나누어 교차 검증하는 kFold Cross Validation은 기존 train, valid 로 나누어 한번만 검증하던 프로세스에서 조금 더 엄격하게 모델을 검증하는 방법입니다. 
- Sklearn의 kFold는 데이터셋의 인덱스로 Fold를 정의하므로 기존에 사용하던 DataSet이 아닌 인덱스로 생성한 Subset을 사용합니다. 이번 실습 자료에서는 매 이터레이션마다 생성한 SubSet 객체를 활용해 DataLoader를 반환해 학습 및 검증에 사용하게 됩니다.

### Model Parameter Setting

In [ ]:
# -- parameters
img_root = '/mnt/ssd/data/mask_final/train/images'  # 학습 이미지 폴더의 경로
label_path = '/mnt/ssd/data/mask_final/train/train.csv'  # 학습 메타파일의 경로

model_name = "VGG19"  # 모델 이름
use_pretrained = True  # pretrained-model의 사용 여부
freeze_backbone = False  # classifier head 이 외 부분을 업데이트되지 않게 할 것인지 여부

batch_size = 64
num_workers = 4
num_classes = 18

num_epochs = 100  # 학습할 epoch의 수
lr = 1e-4
lr_decay_step = 10
criterion_name = 'cross_entropy' # loss의 이름

train_log_interval = 20  # logging할 iteration의 주기
name = "02_vgg"  # 결과를 저장하는 폴더의 이름

# -- settings
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

### DataLoader
- index를 사용한 Dataloader 정의
- getDataloader 함수 설명
    1. Pytorch Dataset, train 인덱스, valid 인덱스, batch size를 전달받아 Train, Valid DataLoader 객체를 반환합니다.
    2. torch.utils.data.Subset 객체는 데이터셋과 해당 데이터셋의 인덱스를 전달받아 Subset 객체를 생성합니다. 생성한 Subset 객체를 사용해 DataLoader 객체를 반환합니다.

In [ ]:
def getDataloader(dataset, train_idx, valid_idx, batch_size, num_workers):
    train_set = torch.utils.data.Subset(dataset,
                                        indices=train_idx)
    val_set   = torch.utils.data.Subset(dataset,
                                        indices=valid_idx)
    val_set.dataset.set_phase("test")

    train_loader = torch.utils.data.DataLoader(
        train_set,
        batch_size=batch_size,
        num_workers=num_workers,
        drop_last=True,
    )

    val_loader = torch.utils.data.DataLoader(
        val_set,
        batch_size=batch_size,
        num_workers=num_workers,
        drop_last=True,
    )

    return train_loader, val_loader

### Stratified k-Fold
1. k를 나타내는 n_splits 값을 설정해 StratifiedKFold 객체를 준비합니다. 
2. skf.split(x, y) 메소드를 사용해 데이터셋의 인덱스를 얻어내고, 라벨(y)에 해당하는 dataset.labels를 기준으로 Stratified를 진행합니다. 
3. 매 이터레이션마다 반환받은 인덱스를 사용해 getDataloader 함수에 전달하여 DataLoader를 생성합니다.
4. 나머지 학습 프로세스는 이전 강의에서 사용한 프로세스와 동일하게 진행합니다.

In [ ]:
dataset = MaskMultiClassDataset(img_root, label_path, 'train')

In [ ]:
os.makedirs(os.path.join(os.getcwd(), 'results', name), exist_ok=True)

n_splits = 5
skf = StratifiedKFold(n_splits=n_splits)

counter = 0
patience = 10
accumulation_steps = 2
best_val_acc = 0
best_val_loss = np.inf
for i, (train_idx, valid_idx) in enumerate(skf.split(dataset.image_paths, dataset.labels)):
    train_loader, val_loader = getDataloader(dataset, train_idx, valid_idx, batch_size, num_workers)

    # -- model
    if model_name == "AlexNet":
        model = AlexNet(num_classes=num_classes, pretrained=use_pretrained, freeze=freeze_backbone).to(device)
    else:
        model = VGG19(num_classes=num_classes, pretrained=use_pretrained, freeze=freeze_backbone).to(device)

    # -- loss & metric
    criterion = create_criterion(criterion_name)
    train_params = [{'params': getattr(model.net, 'features').parameters(), 'lr': lr / 10, 'weight_decay':5e-4},
                    {'params': getattr(model.net, 'classifier').parameters(), 'lr': lr, 'weight_decay':5e-4}]
    optimizer = Adam(train_params)
    scheduler = StepLR(optimizer, lr_decay_step, gamma=0.5)

    # -- logging
    logger = SummaryWriter(log_dir=f"results/cv{i}_{name}")
    for epoch in range(num_epochs):
        # train loop
        model.train()
        loss_value = 0
        matches = 0
        for idx, train_batch in enumerate(train_loader):
            inputs, labels = train_batch
            inputs = inputs.to(device)
            labels = labels.to(device)

            outs = model(inputs)
            preds = torch.argmax(outs, dim=-1)
            loss = criterion(outs, labels)

            loss.backward()
            
             # -- Gradient Accumulation
            if (idx+1) % accumulation_steps == 0:
                optimizer.step()
                optimizer.zero_grad()

            loss_value += loss.item()
            matches += (preds == labels).sum().item()
            if (idx + 1) % train_log_interval == 0:
                train_loss = loss_value / train_log_interval
                train_acc = matches / batch_size / train_log_interval
                current_lr = scheduler.get_last_lr()
                print(
                    f"Epoch[{epoch}/{num_epochs}]({idx + 1}/{len(train_loader)}) || "
                    f"training loss {train_loss:4.4} || training accuracy {train_acc:4.2%} || lr {current_lr}"
                )

                loss_value = 0
                matches = 0

        scheduler.step()

        # val loop
        with torch.no_grad():
            print("Calculating validation results...")
            model.eval()
            val_loss_items = []
            val_acc_items = []
            for val_batch in val_loader:
                inputs, labels = val_batch
                inputs = inputs.to(device)
                labels = labels.to(device)

                outs = model(inputs)
                preds = torch.argmax(outs, dim=-1)

                loss_item = criterion(outs, labels).item()
                acc_item = (labels == preds).sum().item()
                val_loss_items.append(loss_item)
                val_acc_items.append(acc_item)

            val_loss = np.sum(val_loss_items) / len(val_loader)
            val_acc = np.sum(val_acc_items) / len(valid_idx)

            # Callback1: validation accuracy가 향상될수록 모델을 저장합니다.
            if val_loss < best_val_loss:
                best_val_loss = val_loss
            if val_acc > best_val_acc:
                print("New best model for val accuracy! saving the model..")
                torch.save(model.state_dict(), f"results/{name}/{epoch:03}_accuracy_{val_acc:4.2%}.ckpt")
                best_val_acc = val_acc
                counter = 0
            else:
                counter += 1
            # Callback2: patience 횟수 동안 성능 향상이 없을 경우 학습을 종료시킵니다.
            if counter > patience:
                print("Early Stopping...")
                break


            print(
                f"[Val] acc : {val_acc:4.2%}, loss: {val_loss:4.2} || "
                f"best acc : {best_val_acc:4.2%}, best loss: {best_val_loss:4.2}"
            )